In [1]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

In [2]:
terminal_output = open('day7_input.txt').read().splitlines()

In [3]:
terminal_output

['$ cd /',
 '$ ls',
 'dir a',
 '14848514 b.txt',
 '8504156 c.dat',
 'dir d',
 '$ cd a',
 '$ ls',
 'dir e',
 '29116 f',
 '2557 g',
 '62596 h.lst',
 '$ cd e',
 '$ ls',
 '584 i',
 '$ cd ..',
 '$ cd ..',
 '$ cd d',
 '$ ls',
 '4060174 j',
 '8033020 d.log',
 '5626152 d.ext',
 '7214296 k']

In [4]:
terminal_output[3]

'14848514 b.txt'

In [5]:
def ls_statement_content(index, terminal_output):
    items_found = {}    

    if terminal_output[index] == "$ ls":
        #print(i, "checking ls statement")  
        for file_index in range(index+1,  len(terminal_output)):
            if terminal_output[file_index][0] == "$":
                #print("...new command found... exit")
                break
            else:
                #print("---- file", terminal_output[file_index])
                if terminal_output[file_index][0:3] == "dir":
                    directory = terminal_output[file_index][4]
                    items_found[directory] = {}
                else:
                    file_name = re.findall("[\D]+", terminal_output[file_index])[0].strip()
                    file_size = int(re.findall("[0-9]+", terminal_output[file_index])[0])
                    #print(file_name, file_size)
                    items_found[file_name] =  file_size
                    
        return items_found
    
def merge_nested(dict1, dict2):
    for key, value in dict1.items():
        #print(key,value)
        try:
            if key in list(dict2.keys())[0]:
                #print("continue with merging")
                dict1[key].update(dict2[key])
            else:
                merge_nested(dict1[key], dict2) 
            #break
        except:
            continue
            #print("no key found in this branch")
    return dict1

In [38]:
directory_structure = {"/":{}}

current_directory = "/"
for i, line in enumerate(terminal_output):
    #print("**** checking line", i, line)
    if line == "$ ls":
        print(ls_statement_content(i, terminal_output))
        new_folder = {current_directory: ls_statement_content(i, terminal_output)}
        print("new_folder", new_folder)
        print("current directory", current_directory, "--", "directory_struct", directory_structure)
        directory_structure = merge_nested(directory_structure, new_folder)
    
    elif re.search("cd [a-z]", line):
        print("opening directory", line[5])
        current_directory = line[5]

directory_structure

{'a': {}, 'b.txt': 14848514, 'c.dat': 8504156, 'd': {}}
new_folder {'/': {'a': {}, 'b.txt': 14848514, 'c.dat': 8504156, 'd': {}}}
current directory / -- directory_struct {'/': {}}
opening directory a
{'e': {}, 'f': 29116, 'g': 2557, 'h.lst': 62596}
new_folder {'a': {'e': {}, 'f': 29116, 'g': 2557, 'h.lst': 62596}}
current directory a -- directory_struct {'/': {'a': {}, 'b.txt': 14848514, 'c.dat': 8504156, 'd': {}}}
opening directory e
{'i': 584}
new_folder {'e': {'i': 584}}
current directory e -- directory_struct {'/': {'a': {'e': {}, 'f': 29116, 'g': 2557, 'h.lst': 62596}, 'b.txt': 14848514, 'c.dat': 8504156, 'd': {}}}
opening directory d
{'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}
new_folder {'d': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}
current directory d -- directory_struct {'/': {'a': {'e': {'i': 584}, 'f': 29116, 'g': 2557, 'h.lst': 62596}, 'b.txt': 14848514, 'c.dat': 8504156, 'd': {}}}


{'/': {'a': {'e': {'i': 584}, 'f': 29116, 'g': 2557, 'h.lst': 62596},
  'b.txt': 14848514,
  'c.dat': 8504156,
  'd': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}}

In [39]:
for key, val in directory_structure.items():
    print(key)

/


In [40]:
directory_size = 0

def check_directory_size(directory, directory_size):
    for key, val in directory.items():
        #print(key, val)
        if isinstance(val, int):
            directory_size += val
            #print("increasing memory count", key, directory_size)
        else:
            #print("moving forward", key,directory_size)
            directory_size = check_directory_size(directory[key], directory_size)
    
    return directory_size


In [41]:
check_directory_size(directory_structure, 0)

48381165

In [45]:
directory_structure

{'/': {'a': {'e': {'i': 584}, 'f': 29116, 'g': 2557, 'h.lst': 62596},
  'b.txt': 14848514,
  'c.dat': 8504156,
  'd': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}}

In [ ]:
dict1 = {'d': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}

In [46]:
check_directory_size(dict1, 0)

24933642

In [53]:
def check_all_directories(directory_structure):
    directory_check = directory_structure.copy()
    for key, val in directory_check.items():
        if isinstance(val, dict):
            print(key, val)
            print(check_directory_size(directory_check, 0))
            check_all_directories(directory_check[key])  

In [54]:
check_all_directories(directory_structure)

/ {'a': {'e': {'i': 584}, 'f': 29116, 'g': 2557, 'h.lst': 62596}, 'b.txt': 14848514, 'c.dat': 8504156, 'd': {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}}
48381165
a {'e': {'i': 584}, 'f': 29116, 'g': 2557, 'h.lst': 62596}
48381165
e {'i': 584}
94853
d {'j': 4060174, 'd.log': 8033020, 'd.ext': 5626152, 'k': 7214296}
48381165


In [ ]:
def return_dir_size(
    
for key, value in directory_structure.items()

In [56]:
pip install anytree

Looking in indexes: https://artifactory.sofa.dev/artifactory/api/pypi/pypi-remote/simple, https://artifactory.sofa.dev/artifactory/api/pypi/pypi-local/simple, https://artifactory.sofa.dev/artifactory/api/pypi/disc-ddp-pypi-local/simple
     -------------------------------------- 41.7/41.7 kB 983.7 kB/s eta 0:00:00

[notice] A new release of pip available: 22.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [58]:

# --- Day 7: No Space Left On Device ---

from anytree import Node, RenderTree
import re

curr_dir = None
with open("day7_input.txt") as fp:
    for line in fp:
        match_cd = re.search(r"\$ cd (.+)", line)
        match_file = re.search(r"(\d+) (.+)", line)
        if match_cd:
            arg = match_cd.group(1)
            name = f"{arg} (dir)"
            curr_dir = Node(name, parent=curr_dir) if arg != ".." else curr_dir.parent
        elif match_file:
            size, name = match_file.groups()
            name = f"{name} (file, size={size})"
            Node(name, parent=curr_dir)

for pre, _, node in RenderTree(curr_dir.root):
    print(pre + node.name)

sizes = {}
for leaf in curr_dir.root.leaves:
    size = int(leaf.name.split("=")[-1][:-1])
    for a in leaf.ancestors:
        path = "/".join([x.name for x in a.path])
        sizes[path] = size if path not in sizes else sizes[path] + size

disk_space_used = 70_000_000 - sizes["/ (dir)"]
disk_space_to_clean = 30_000_000 - disk_space_used

part_one = sum([x for x in sizes.values() if x <= 100_000])
part_two = min([x for x in sizes.values() if x >= disk_space_to_clean])

print("\nPart 1:", part_one, "\nPart 2:", part_two)

/ (dir)
├── ghpqs (file, size=307728)
├── whhj.pqt (file, size=120579)
├── dfmhjhd (dir)
│   ├── ngtqtf (file, size=301486)
│   └── wfgqtw.sqr (file, size=13488)
├── hztjntff (dir)
│   ├── ftq.cjn (file, size=288227)
│   ├── hwtj (file, size=176977)
│   ├── nzdgz.mpw (file, size=234858)
│   ├── rhs.mbd (file, size=157857)
│   ├── cwsf (dir)
│   │   ├── mvfclz.srm (file, size=272534)
│   │   ├── nldtb (file, size=226182)
│   │   ├── qnwzs (file, size=252294)
│   │   └── msr (dir)
│   │       └── hwtj (file, size=272198)
│   └── sthqhrc (dir)
│       └── vzntwhzl.htf (file, size=45847)
├── rvstq (dir)
│   ├── bqg (file, size=9707)
│   ├── frcqrdm (file, size=296975)
│   ├── hztjntff (file, size=232225)
│   ├── ldmtcrq.dcs (file, size=29794)
│   ├── wrmm.cjh (file, size=28366)
│   ├── nzdgz (dir)
│   │   ├── sdnvpt (file, size=117527)
│   │   ├── fplqm (dir)
│   │   │   ├── mdcnwnng (file, size=216299)
│   │   │   ├── nzdgz (file, size=109253)
│   │   │   ├── cwsf (dir)
│   │   │   │   └─

# Part 1

In [51]:
find_marker_message(datastream, 5)

Marker found at char:


1930

# Part 2

In [50]:
find_marker_message(datastream, 14)

Marker found at char:


3298